In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('TES_patents.csv')

In [3]:
applicants_raw = df['출원인']

In [4]:
#일차적으로 겹치는 항목 제거
applicants = applicants_raw.unique().tolist()

In [5]:
len(applicants)

4648

In [6]:
from fuzzywuzzy import process
import pandas as pd

In [7]:
threshold = 80
unique_applicants = set()

#일정수준 이상 유사한 단어 제외하고 리스트 생성 
for applicant in applicants:
    # unique_applicants 집합이 비어 있으면 첫 번째 항목을 직접 추가
    if not isinstance(applicant, str):
        continue
    
    if not unique_applicants:
        unique_applicants.add(applicant)
        continue

    # 모든 가능한 매칭과 유사도를 가져옴
    matches = process.extract(applicant, unique_applicants, limit=None)

    # 가장 높은 유사도를 가진 항목 찾기
    if not matches or max(matches, key=lambda x: x[1])[1] < threshold:
        unique_applicants.add(applicant)

In [8]:
len(unique_applicants)

894

In [17]:
import requests
from bs4 import BeautifulSoup
import itertools
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [10]:
big = [
    "Saudi Arabian Oil", "Shell", "Chevron", "SK innovation", 
    "China Petroleum", "BP p.l.c", "University", 
    "Petrobras", "Eni S.p.A", "TotalEnergies", "PetroChina", 
    "Equinor ASA", "Suncor Energy", "Exxon Mobil", 
    "schlumberger"
]

In [11]:
companies = big+list(unique_applicants)

In [19]:
results = []
utc=pytz.UTC
two_years_ago = datetime.now(utc) - timedelta(days=2*365)

In [13]:
company_pairs = list(itertools.combinations(companies, 2))

In [ ]:
# 각 쌍에 대해 검색 및 크롤링
for pair in company_pairs:
    query = f'"{pair[0]}" "{pair[1]}"'
    url = f'https://news.google.com/search?q={query}%20when%3A1y&hl=en-US&gl=US&ceid=US:en'
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 제목 및 날짜 찾기
    for item in soup.find_all('article'):
        title = item.find('h3').get_text() if item.find('h3') else None
        date_str = item.find('time')['datetime'] if item.find('time') else None

        # 날짜 파싱 및 검증
        if date_str:
            article_date = datetime.fromisoformat(date_str)
            if article_date >= two_years_ago and pair[0] in title and pair[1] in title:
                results.append({'Company1': pair[0], 'Company2': pair[1], 'Title': title, 'Date': article_date})